In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense

In [2]:
model = Sequential()                                                                # keras modeli
model.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))  # evrisim katmani (64, 64, 3) RGB ve 64x64px indirge
model.add(MaxPooling2D(pool_size = (2, 2) ))                                        # pooling katmani max algoritmasi    
model.add(Convolution2D(32, 3, 3, activation = 'relu'))                             # evrisim katmani
model.add(MaxPooling2D(pool_size = (2, 2) ))                                        # pooling katmani max algoritmasi
model.add(Flatten())                                                                # Flatten islemi
model.add(Dense(128, activation = 'relu'))                                          # YSA 128 girdi
model.add(Dense(1, activation = 'sigmoid'))                                         # YSA 1 cikti

In [3]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

**shear_range: Float. Shear Intensity (Shear angle in counter-clockwise direction in degrees) | Sair diyor ki; float deger alir. kesme yogunlugu yani oransal girili ve saat yonunun tersine shear acisi alir**

**zoom_range: Float or [lower, upper]. Range for random zoom. If a float, [lower, upper] = [1-zoom_range, 1+zoom_range].**

-> **object** : the Keras Object model.

-> **generator** : a generator whose output must be a list of the form:
                      - (inputs, targets)    
                      - (input, targets, sample_weights)
a single output of the generator makes a single batch and hence all arrays in the list 
must be having the length equal to the size of the batch. The generator is expected 
to loop over its data infinite no. of times, it should never return or exit.

-> **steps_per_epoch** : it specifies the total number of steps taken from the generator
   as soon as one epoch is finished and next epoch has started. We can calculate the value
of steps_per_epoch as the total number of samples in your dataset divided by the batch size.

-> **Epochs** : an integer and number of epochs we want to train our model for.

-> **Verbose** : specifies verbosity mode(0 = silent, 1= progress bar, 2 = one line per epoch).

-> **callbacks** : a list of callback functions applied during the training of our model.

-> **validation_data** can be either:
                      - an inputs and targets list
                      - a generator
                      - an inputs, targets, and sample_weights list which can be used to evaluate
                        the loss and metrics for any model after any epoch has ended.

-> **validation_steps** :only if the validation_data is a generator then only this argument
can be used. It specifies the total number of steps taken from the generator before it is 
stopped at every epoch and its value is calculated as the total number of validation data points
in your dataset divided by the validation batch size.

In [34]:
from keras.preprocessing.image import ImageDataGenerator
trainDatagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2, # yana bukme     
                                  zoom_range = 0.2,       
                                  horizontal_flip = True)

testDatagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,      
                                  zoom_range = 0.2,       
                                  horizontal_flip = True) 

trainingSet = trainDatagen.flow_from_directory(
    'veriler/training_set',
    target_size = (64, 64),
    batch_size = 2,
    class_mode = 'binary'
)

testSet = trainDatagen.flow_from_directory(  # bu bize (x, y) don durur bu da -> (batch_size, **target_size, channels) ve y
    'veriler/test_set',
    target_size = (64, 64),
    batch_size = 2,
    class_mode = 'binary'
)

model.fit_generator(
    generator = trainingSet,                          # okudugumuz degeri   
    steps_per_epoch = len(trainingSet),               # her epoch da okunacak ornek
    epochs = 1,                                      # kac epoch istedigimiz
    validation_data = testSet,                        # dogrulama  verileri
    validation_steps = len(testSet)/2                 #  
)

Found 2662 images belonging to 2 classes.
Found 203 images belonging to 2 classes.
Epoch 1/10
1331/1331 [==============================] - 20s 15ms/step - loss: 0.1970 - accuracy: 0.9038 - val_loss: 0.1894 - val_accuracy: 0.9216
Epoch 2/10
1331/1331 [==============================] - 19s 14ms/step - loss: 0.1886 - accuracy: 0.9053 - val_loss: 0.3676 - val_accuracy: 0.7745
Epoch 3/10
1331/1331 [==============================] - 19s 15ms/step - loss: 0.1809 - accuracy: 0.9106 - val_loss: 0.3401 - val_accuracy: 0.8235
Epoch 4/10
1331/1331 [==============================] - 21s 16ms/step - loss: 0.1785 - accuracy: 0.9106 - val_loss: 0.2851 - val_accuracy: 0.8627
Epoch 5/10
1331/1331 [==============================] - 21s 16ms/step - loss: 0.1915 - accuracy: 0.9061 - val_loss: 0.2715 - val_accuracy: 0.8431
Epoch 6/10
1331/1331 [==============================] - 18s 14ms/step - loss: 0.1902 - accuracy: 0.9061 - val_loss: 0.1873 - val_accuracy: 0.9510
Epoch 7/10
1331/1331 [===================

In [35]:
import pandas as pd 
import numpy as np

testSet.reset()
pred = model.predict_generator(testSet, verbose = 1)

pred[pred > 0.5] = 1
pred[pred <= 0.5] = 0

testLabel = []
for i in range(len(testSet)):
    testLabel.extend(np.array(testSet[i][1])) # yukarida anlattim y kismi burada doner. (In the section directoryiterator)

#print(testSet.filenames)
d = {'dosyaIsimleri' : testSet.filenames, 'tahminler' : pred.tolist() , 'test' : testLabel}
cikti = pd.DataFrame(d)
cikti

102/102 [==============================] - 1s 14ms/step


,dosyaIsimleri,tahminler,test
0,erkek\AbdA_00001_m_31_i_fr_nc_no_2016_2_e0_nl_...,[1.0],0.0
1,erkek\AbdA_00002_m_31_i_fr_nc_sr_2016_2_e0_nl_...,[0.0],0.0
2,erkek\AbdA_00003_m_31_i_fr_nc_hp_2016_2_e0_nl_...,[0.0],0.0
3,erkek\AbdA_00004_m_31_i_fr_nc_hp_2016_2_e0_nl_...,[0.0],0.0
4,erkek\AbdA_00005_m_31_i_fr_nc_hp_2016_2_e0_nl_...,[0.0],0.0
...,...,...,...
198,kadin\HadM_00817_f_23_i_nf_nc_hp_2014_1_e0_nl_...,[1.0],1.0
199,kadin\HadM_00818_f_25_o_nf_nc_no_2016_1_e0_nl_...,[0.0],0.0
200,kadin\HadM_00819_f_25_o_nf_nc_no_2016_1_e0_nl_...,[0.0],0.0
201,kadin\HebD_00821_f_28_i_nf_cr_sd_2016_1_e0_nl_...,[0.0],0.0


In [44]:
"""**Confision Matrix**"""
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(testLabel, pred)
cm

array([[121,  30],
       [ 38,  14]], dtype=int64)

In [46]:
(cm[0, 0] + cm[1, 1])/sum(cm.reshape(-1, 1))

array([0.66502463])